In [19]:
import base64
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve API key from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)

# Initialize an empty dictionary to store responses
responses = {}

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Specify the folder containing the images
image_folder_path = "invoices"

# Loop through all image files in the folder
for filename in os.listdir(image_folder_path):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(image_folder_path, filename)
        base64_image = encode_image(image_path)

        # Make the API call for each image
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": "Retrieve invoice_number, date_of_issue, seller_info, client_info,"
                                    "invoice_items_table, invoice_summary_table. Response must be in JSON format",
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            },
                        },
                    ],
                }
            ],
        )

        # Save the JSON response in the dictionary with the filename as the key
        responses[filename] = response.choices[0].to_dict()
        break

# Output the dictionary with file names and their responses
responses

{'Kreditnota_test_firma.png': {'finish_reason': 'stop',
  'index': 0,
  'logprobs': None,
  'message': {'content': '```json\n{\n  "invoice_number": "1",\n  "date_of_issue": "16-08-2023",\n  "seller_info": {\n    "name": "Testfirma",\n    "address": "Testvej 3, 3500 Værløse",\n    "contact": {\n      "email": "test@minuba.dk",\n      "phone": "70702037",\n      "CVR": "23456267"\n    }\n  },\n  "client_info": {\n    "name": "Test Testesen",\n    "address": "Testvej 1, 3500 Værløse",\n    "customer_number": "01020304",\n    "customer_CVR": "12345678"\n  },\n  "invoice_items_table": [\n    {\n      "description": "Montør (timer)",\n      "quantity": -1.5,\n      "unit_price": 395.00,\n      "total": -592.50\n    },\n    {\n      "description": "Servicevogn",\n      "quantity": -1.5,\n      "unit_price": 65.00,\n      "total": -97.50\n    },\n    {\n      "description": "KOMB.SIGN M/I/FØ",\n      "quantity": -1,\n      "unit_price": 2325.00,\n      "total": -2325.00\n    },\n    {\n      "

In [20]:
import json
raw_content = responses['Kreditnota_test_firma.png']['message']['content']
json_content = raw_content.strip('```json\n').strip('\n```')

# Load the JSON content
parsed_json = json.loads(json_content)

parsed_json

{'invoice_number': '1',
 'date_of_issue': '16-08-2023',
 'seller_info': {'name': 'Testfirma',
  'address': 'Testvej 3, 3500 Værløse',
  'contact': {'email': 'test@minuba.dk',
   'phone': '70702037',
   'CVR': '23456267'}},
 'client_info': {'name': 'Test Testesen',
  'address': 'Testvej 1, 3500 Værløse',
  'customer_number': '01020304',
  'customer_CVR': '12345678'},
 'invoice_items_table': [{'description': 'Montør (timer)',
   'quantity': -1.5,
   'unit_price': 395.0,
   'total': -592.5},
  {'description': 'Servicevogn',
   'quantity': -1.5,
   'unit_price': 65.0,
   'total': -97.5},
  {'description': 'KOMB.SIGN M/I/FØ',
   'quantity': -1,
   'unit_price': 2325.0,
   'total': -2325.0},
  {'description': 'SÆDE I/FØ SIGN Q',
   'quantity': -1,
   'unit_price': 734.0,
   'total': -734.0},
  {'description': 'TILS.L. 1/2',
   'quantity': -1,
   'unit_price': 52.0,
   'total': -52.0},
  {'description': 'LYNBETON CX-5 1 KG 27 1569 010 - I POSER',
   'quantity': -0.5,
   'unit_price': 61.0,
  